## Libraries and Auth

In [ ]:
!pip install discogs-client
!pip install tensorflow
!pip install colorthief opencv-python numpy
!pip install webcolors

In [1]:
# Import the discogs_client module
import discogs_client
from config import CONSUMER_KEY, CONSUMER_SECRET  # Import your credentials

def authenticate_discogs():
    """Authenticate with the Discogs API and return the client."""
    # Initialize Discogs client
    client = discogs_client.Client(user_agent='RecordCollectionApp/1.0', consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET)

    try:
        # Step 1: Get authorization URL
        auth_url = client.get_authorize_url()
        print(f"Visit this URL to authorize the app: {auth_url}")

        # Step 2: Prompt user for verifier code from Discogs
        verifier = input("Enter the code from Discogs: ")

        # Step 3: Exchange verifier for an access token
        access_token, access_secret = client.get_access_token(verifier)
        print("Authentication successful!")

    except Exception as e:  # Catch all exceptions
        print("Authentication failed:", e)
        return None

    return client

Authenticate the Session:

In [2]:
client = authenticate_discogs()

Visit this URL to authorize the app: ('rQGBtpVGKrEmUhdikaTVKEAKkfAxoegQcOdgAiSb', 'MWpUiLKZwTzqsmYkGbfrjhZfDEubfWQBwBsXMnmG', 'https://www.discogs.com/oauth/authorize?oauth_token=rQGBtpVGKrEmUhdikaTVKEAKkfAxoegQcOdgAiSb')
Enter the code from Discogs: jFVnqHunsL
Authentication successful!


## My Collection

In [3]:
def get_collection_by_genre(client, print_collection=False):
    """Fetch and return albums from the user's collection organized by genre without duplicates.

    Args:
        client: The Discogs API client.
        print_collection (bool): If True, prints the collection; otherwise, it does not.

    Returns:
        dict: A dictionary of genres and their respective albums.
    """
    try:
        user = client.identity()  # Get your user information
        folders = user.collection_folders  # Get the user's collection folders

        genre_dict = {}  # Dictionary to hold genres and their respective albums
        album_set = set()  # Set to hold unique albums

        for folder in folders:
            releases = folder.releases  # Get the releases in the folder
            for release in releases:
                release_details = release.release  # Fetch the release details
                genres = release_details.genres  # Get genres from the release
                for genre in genres:
                    if genre not in genre_dict:
                        genre_dict[genre] = []
                    album_identifier = f"{release_details.title} by {release_details.artists[0].name}"
                    if album_identifier not in album_set:
                        album_set.add(album_identifier)
                        genre_dict[genre].append(album_identifier)

        # Print each genre and its albums if print_collection is True
        if print_collection:
            for genre, albums in genre_dict.items():
                print(f"\nGenre: {genre}")
                for album in albums:
                    print(f"  - {album}")

        return genre_dict

    except Exception as e:
        print("Error retrieving collection:", e)
        return None

Display the Collection:

In [4]:
if client:
    collection_dict = get_collection_by_genre(client, print_collection=True)


Genre: Rock
  - Beat Crazy by Joe Jackson Band
  - Even In The Quietest Moments... by Supertramp
  - Everybody Else Is Doing It, So Why Can't We? by The Cranberries
  - Waiting to Spill by The Backseat Lovers
  - Duke by Genesis
  - Charm by Clairo (2)
  - Where The Light Is (John Mayer Live In Los Angeles) by John Mayer
  - Stranger In The Alps by Phoebe Bridgers
  - Punisher by Phoebe Bridgers
  - A La Sala by Khruangbin
  - Being Funny In A Foreign Language by The 1975
  - Imperial Bedroom by Elvis Costello & The Attractions
  - Kansas Anymore by Role Model (2)
  - The Record by Boygenius
  - Blood Bank by Bon Iver
  - Parachutes by Coldplay
  - NFR! by Lana Del Rey
  - Take The Sadness Out Of Saturday Night by Bleachers
  - Preservation Act 1 by The Kinks
  - Blue by Joni Mitchell
  - The Dance by Fleetwood Mac
  - Stick Season (We’ll All Be Here Forever) by Noah Kahan
  - Oncle Jazz by Men I Trust
  - Hozier by Hozier
  - Atlanta Millionaires Club by Faye Webster
  - Bloom by Bea

## Recommendation Algorithm

In [5]:
import random
from textblob import TextBlob
from discogs_client import Client  # Make sure to initialize your Discogs API client as needed

# Helper functions
def analyze_vibe(vibe_description):
    """Analyze vibe using sentiment analysis and map it to mood-based genres."""
    vibe_blob = TextBlob(vibe_description)
    sentiment = vibe_blob.sentiment.polarity  # Sentiment score between -1 (negative) and 1 (positive)

    # Sentiment-based genre mapping
    if sentiment > 0.3:
        sentiment_genres = ["Pop", "Electronic", "Funk", "Reggae"]
    elif sentiment < -0.3:
        sentiment_genres = ["Blues", "Classical", "Ambient", "Folk"]
    else:
        sentiment_genres = ["Jazz", "Hip Hop", "Lo-Fi", "Indie"]

    return sentiment_genres

def map_activity_to_genre(activity):
    """Map specific activities to genres."""
    activity_genre_map = {
        "social gathering": ["Pop", "Dance", "Jazz", "Funk"],
        "game night": ["Rock", "Hip Hop", "Funk", "Electronic"],
        "cooking": ["Jazz", "Classical", "Bossa Nova", "Funk"],
        "creative activities": ["Indie", "Ambient", "Electronic", "Pop"],
        "reading": ["Classical", "Ambient", "Lo-Fi", "Jazz", "Indie"],
        "wine/drinks": ["Jazz", "Blues", "Soul", "Classical"],
        "just listening": ["Indie", "Alternative", "Ambient", "Folk"],
    }
    return activity_genre_map.get(activity.lower(), ["Indie", "Jazz", "Pop"])  # Default genres

def determine_genre(activity_description, vibe_description):
    """Map the user's activity and vibe to a suggested genre list."""
    # Step 1: Get activity-based genre suggestions
    activity_genres = map_activity_to_genre(activity_description)

    # Step 2: Get sentiment-based genre suggestions
    sentiment_genres = analyze_vibe(vibe_description)

    # Step 3: Combine both genre lists
    combined_genres = list(set(activity_genres) | set(sentiment_genres))  # Union of both lists

    return combined_genres  # Return the combined genres instead of one selected genre

# Primary music recommendation function
def music_recommendation(client):
    """Prompts the user for music preferences and recommends albums based on genre analysis."""
    # Step 1: Fetch collection by genre
    genre_dict = get_collection_by_genre(client)
    if not genre_dict:
        print("No albums found in your collection.")
        return

    print("Welcome to the Music Recommendation Assistant!")

    # Step 2: Prompt user input for session type
    session_type = input("Is this for a group event (g) or a solo/intimate session (s)? ").strip().lower()

    if session_type == 'g':
        activity_description = input("Choose group activity: Social Gathering, Game Night, Cooking, Creative Activities: ").strip().lower()
    elif session_type == 's':
        activity_description = input("Choose solo activity: Reading, Cooking, Wine/Drinks, Game Night, Creative Activities, Just Listening: ").strip().lower()
    else:
        print("Invalid input. Please try again.")
        return

    # Step 3: Prompt for vibe
    vibe_description = input("Describe the vibe you're aiming for: ")

    # Step 4: Determine genres based on activity and vibe
    selected_genres = determine_genre(activity_description, vibe_description)

    # Step 5: Recommend albums from the selected genres
    recommended_albums = set()  # Use a set to avoid duplicates

    # Loop through the selected genres to get albums
    for genre in selected_genres:
        albums_in_genre = genre_dict.get(genre, [])
        if albums_in_genre:
            recommended_albums.update(random.sample(albums_in_genre, min(1, len(albums_in_genre))))  # Get one album per genre

    # Step 6: Check if we have recommendations
    if not recommended_albums:
        print("No albums found in any genres.")
        return

    # Step 7: Output recommendations
    print(f"\nFor your {session_type} event ({activity_description.capitalize()}) with a '{vibe_description}' vibe, we recommend:")
    for album in recommended_albums:
        print(f"Album: {album}")

# Example usage
# client = Client('YOUR_USER_AGENT')  # Initialize the Discogs client with your user agent and token

Generate Recommendations:

In [6]:
music_recommendation(client)

Welcome to the Music Recommendation Assistant!
Is this for a group event (g) or a solo/intimate session (s)? g
Choose group activity: Social Gathering, Game Night, Cooking, Creative Activities: cooking
Describe the vibe you're aiming for: upbeat

For your g event (Cooking) with a 'upbeat' vibe, we recommend:
Album: MM..Food by MF Doom
Album: Come Away With Me by Norah Jones


.

## Album Art Fetching

In [7]:
client = authenticate_discogs()

# Global variable to hold the album cover URL
album_cover_url = ""

def get_album_cover(album_title, client):
    """Retrieve the cover art for a specific album and store it in a global variable."""
    global album_cover_url  # Declare the use of the global variable

    try:
        # Search for the album using the authenticated client
        results = client.search(album_title, type='release')
        if not results:
            print(f"No results found for '{album_title}'.")
            return None

        # Get the first result
        album = results[0]

        # Extract the cover art URL from the images attribute
        if album.images:
            album_cover_url = album.images[0]['uri']  # Get the URL of the first image
            print(f"Fetched album cover URL for '{album_title}': {album_cover_url}")
            return album_cover_url
        else:
            print(f"No cover images found for '{album_title}'.")
            return None

    except Exception as e:
        print("Error retrieving album cover art:", e)
        return None

Visit this URL to authorize the app: ('JMviqjRwDUXyuVailfTbxbkLAdSUAfCUIXgyiDlW', 'UQurTINVlEknYuYSFaBTiFjUbupABsFdiXjtUSIO', 'https://www.discogs.com/oauth/authorize?oauth_token=JMviqjRwDUXyuVailfTbxbkLAdSUAfCUIXgyiDlW')
Enter the code from Discogs: nCzLhoKiSi
Authentication successful!


In [15]:
# Main flow
if __name__ == "__main__":
    if client:  # Check if the client is valid before proceeding
        album_title = input("Enter the album title: ")
        get_album_cover(album_title, client)
    else:
        print("Authentication failed. Please try again.")

Enter the album title: nevermind
Fetched album cover URL for 'nevermind': https://i.discogs.com/uCeQtLv9OSNjC5_AjarCojZNepI9vqcnYeqsImnzXyg/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTcwOTcw/NTEtMTU1NjQ0NDE4/MC03NTA1LmpwZWc.jpeg


.

## Album Art Classification

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import requests
from PIL import Image
from io import BytesIO

# Load the pre-trained MobileNetV2 model
model = tf.keras.applications.MobileNetV2(weights='imagenet')

def preprocess_image(img_url):
    """Load and preprocess the image."""
    # Set custom headers to include User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }

    # Load the image from the URL with custom headers
    response = requests.get(img_url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Error: Unable to fetch image, status code: {response.status_code}")
        return None

    # Check content type
    content_type = response.headers.get('Content-Type')
    print(f"Content type: {content_type}")  # Debug statement

    # Ensure the content type is an image
    if 'image' not in content_type:
        print("Error: URL does not point to an image.")
        return None

    try:
        img = Image.open(BytesIO(response.content))
    except Exception as e:
        print("Error opening image:", e)
        return None

    # Resize the image to the input shape of MobileNetV2
    img = img.resize((224, 224))

    # Convert the image to an array and preprocess it
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

    return img_array

def predict_aesthetic(img_url):
    """Predict the aesthetic category of the album cover."""
    img_array = preprocess_image(img_url)

    # If the image preprocessing failed, return
    if img_array is None:
        return

    # Make a prediction
    predictions = model.predict(img_array)

    # Decode the predictions
    decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=5)[0]

    # Print the top 5 predictions
    for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
        print(f"{i + 1}: {label} ({score:.2f})")

# Example usage
if __name__ == "__main__":
    predict_aesthetic(album_cover_url)

Content type: image/jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1: snorkel (0.30)
2: scuba_diver (0.05)
3: jellyfish (0.04)
4: electric_ray (0.03)
5: hammerhead (0.03)


## Album Cover Color Classification

In [17]:
import requests
from PIL import Image
from io import BytesIO
from colorthief import ColorThief

# Custom color dictionary
COLOR_DICT = {
    'red': (255, 0, 0),
    'green': (0, 128, 0),
    'blue': (0, 0, 255),
    'yellow': (255, 255, 0),
    'cyan': (0, 255, 255),
    'magenta': (255, 0, 255),
    'black': (0, 0, 0),
    'white': (255, 255, 255),
    'gray': (128, 128, 128),
    'orange': (255, 165, 0),
    'purple': (128, 0, 128),
    'pink': (255, 192, 203),
    'brown': (165, 42, 42),
    'lightblue': (173, 216, 230),
    # Add more colors as needed
}

def fetch_image(img_url):
    """Fetch the image from the given URL."""
    # Set custom headers to include User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Gecko/20100101 Firefox/100.0',
        'Accept': 'image/webp,*/*',
        'Referer': 'https://www.example.com'  # Adjust this if necessary
    }

    # Load the image from the URL with custom headers
    response = requests.get(img_url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Error: Unable to fetch image, status code: {response.status_code}")
        return None

    # Check content type
    content_type = response.headers.get('Content-Type')
    print(f"Content type: {content_type}")  # Debug statement

    # Ensure the content type is an image
    if 'image' not in content_type:
        print("Error: URL does not point to an image.")
        return None

    try:
        img = Image.open(BytesIO(response.content))
    except Exception as e:
        print("Error opening image:", e)
        return None

    return img

def rgb_to_color_name(rgb):
    """Convert RGB values to the closest color name using the custom color dictionary."""
    # Normalize the RGB values
    rgb = (rgb[0], rgb[1], rgb[2])  # Ensure it's a tuple

    # Try to get the closest name based on the RGB value
    closest_name = None
    min_distance = float('inf')

    # Iterate over the custom color dictionary
    for name, color_rgb in COLOR_DICT.items():
        # Calculate the Euclidean distance
        distance = ((rgb[0] - color_rgb[0]) ** 2 +
                     (rgb[1] - color_rgb[1]) ** 2 +
                     (rgb[2] - color_rgb[2]) ** 2) ** 0.5

        if distance < min_distance:
            min_distance = distance
            closest_name = name

    return closest_name

def extract_colors(img):
    """Extract colors from the image using ColorThief."""
    # Save the image to a temporary file for ColorThief
    temp_file_path = "temp_image.jpg"
    img.save(temp_file_path)

    # Initialize ColorThief with the image
    color_thief = ColorThief(temp_file_path)

    # Get the dominant color
    dominant_color = color_thief.get_color(quality=1)

    # Build a color palette
    palette = color_thief.get_palette(color_count=6)

    return dominant_color, palette

def analyze_album_cover(img_url):
    """Fetch and analyze the album cover for dominant colors."""
    img = fetch_image(img_url)

    # If the image fetching failed, return
    if img is None:
        return

    # Extract colors from the image
    dominant_color, palette = extract_colors(img)

    # Convert RGB to color names
    dominant_color_name = rgb_to_color_name(dominant_color)
    palette_names = [rgb_to_color_name(color) for color in palette]

    # Print results
    print(f"Dominant Color: {dominant_color} - {dominant_color_name}")
    print("Color Palette:")
    for color, name in zip(palette, palette_names):
        print(f"{color} - {name}")

# Example usage
if __name__ == "__main__":
    analyze_album_cover(album_cover_url)

Content type: image/jpeg
Dominant Color: (30, 79, 134) - gray
Color Palette:
(25, 67, 126) - gray
(49, 198, 223) - cyan
(189, 219, 223) - lightblue
(23, 161, 203) - cyan
(52, 140, 168) - gray
(15, 11, 24) - black


.